In [ ]:
# Created a 3D exoplanet map. Credits to Microsoft Copilot for most of the code.
# To do: add representation of Earth
# 2d: change size of dots, overlay earth map for reference
# Poster: add the starmaps first, with some labels.

import dash
import pandas as pd
import numpy as np
from dash import Dash, dcc, html, Input, Output, State, Dash, no_update
import plotly.graph_objects as go
import plotly.express as px

%run -i assets/lists.ipynb

# imports Planetary systems csv
pl_s = pd.read_csv('assets/Planetary_Systems.csv', comment="#", low_memory=False)
pl_es = pd.read_csv('assets/Planetary_Systems_Estimated.csv', comment="#", low_memory=False)

# imports atmospheric list for one filter.
at_s = pd.read_csv('assets/Atmospheric_Spectroscopy.csv', comment="#", low_memory=False)

solar_planets = pd.read_csv('assets/Solar_Values.csv', comment="#", low_memory=False)

# Dash app
app = Dash(__name__, external_stylesheets=['assets/style.css'])

app.layout = html.Div([
    html.H1("3D Exoplanet \"Starmap\""),

    html.H2("Data filters:"),
    dcc.Checklist(
        id='filter-checklist',
        options=[
            {'label': 'Has atmospheric data (Compare Planets only)', 'value': 'atmoData'},
            {'label': 'Default parameter set', 'value': 'default'},
            {'label': 'No controversial flag', 'value': 'noControv'},
            {'label': 'Water to Metal Density (Compare Planets only)', 'value': 'densRange'},
            {'label': 'In Target Star Catalog', 'value': 'target'},
        ],
        value=['default', 'noControv'],  # Default values to filter by
        className="checkbox-container"
    ),
    html.Div([
        dcc.Checklist(
            id='host-star-toggle',
            options=[{'label': 'Filter by host star: ', 'value': 'enable'}],
            value=[],  # Start unchecked
            className="checkbox-container"
        ),
        dcc.Input(
            id='host-star-input',
            type='text',
            placeholder='Enter host star name',
            debounce=True,
            className="textbox-style"
        ),
    ], className="checkbox-container", style={'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center', 'gap': '10px'}),

    html.Details([
        html.Summary("Planet type (Compare Planets only)"),
        dcc.Checklist(
            id='pltype-checklist',
            options=[
                {'label': 'Terrestrial', 'value': 'terrestrial'},
                {'label': 'Super-Earth', 'value': 'super_earth'},
                {'label': 'Neptune-like', 'value': 'neptune_like'},
                {'label': 'Gas Giant', 'value': 'gas_giant'},
                {'label': 'Unknown', 'value': 'unknown'},
                {'label': 'TBA', 'value': 'tba'},
            ],
            value=['terrestrial', 'super_earth', 'neptune_like', 'gas_giant', 'unknown', 'tba'],  # Default values to filter by
            className="checkbox-container"
        ),
    ]),

    html.Details([
        html.Summary("Discovery method (Compare Planets only)"),
        dcc.Checklist(
            id='discmethod-checklist',
            options=[
                {'label': 'Transit', 'value': 'Transit'},
                {'label': 'Transit Timing Variations', 'value': 'Transit Timing Variations'},
                {'label': 'Eclipse Timing Variations', 'value': 'Eclipse Timing Variations'},
                {'label': 'Orbital Brightness Modulation', 'value': 'Orbital Brightness Modulation'},
                {'label': 'Radial Velocity', 'value': 'Radial Velocity'},
                {'label': 'Astrometry', 'value': 'Astrometry'},
                {'label': 'Imaging', 'value': 'Imaging'},
                {'label': 'Disc Kinematics', 'value': 'Disc Kinematics'},
                {'label': 'Microlensing', 'value': 'Microlensing'},
                {'label': 'Pulsar Timing', 'value': 'Pulsar Timing'},
                {'label': 'Pulsation Timing Variations', 'value': 'Pulsation Timing Variations'},
                {'label': 'Known Since Antiquity', 'value': 'Known Since Antiquity'},
                {'label': 'Unknown', 'value': 'null'},
            ],
            value=['Transit', 'Radial Velocity', 'Imaging', 'Eclipse Timing Variations', 'Microlensing', 'Pulsar Timing', 'Pulsation Timing Variations', 'Orbital Brightness Modulation', 'Transit Timing Variations', 'Astrometry', 'Disc Kinematics', 'Known Since Antiquity', 'null'],  # automatically selects all
            className="checkbox-container"
        ),
    ]),  

    html.Details([
        html.Summary("Discovery locale (Compare Planets only)"),
        dcc.Checklist(
            id='disclocale-checklist',
            options=[
                {'label': 'Space', 'value': 'Space'},
                {'label': 'Ground', 'value': 'Ground'},
                {'label': 'Multiple Locales', 'value': 'Multiple Locales'},
                {'label': 'Unknown', 'value': 'null'},
            ],
            value=['Space', 'Ground', 'Multiple Locales', 'null'],  # Default values to filter by
            className="checkbox-container"
        ),
    ]), 
    
    html.Details([
        html.Summary("Harvard spectral classes"),
        dcc.Checklist(
            id='teff-checklist',
            options=[
                {'label': 'O type star (>33000 K)', 'value': 'O'},
                {'label': 'B type star (10000-33000 K)', 'value': 'B'},
                {'label': 'A type star (7300-10000 K)', 'value': 'A'},
                {'label': 'F type star (6000-7300 K)', 'value': 'F'},
                {'label': 'G type star (5300-6000 K)', 'value': 'G'},
                {'label': 'K type star (3900-5300 K, recommended)', 'value': 'K'},
                {'label': 'M type star (2300-3900 K)', 'value': 'M'},
                {'label': 'L type star (1300-2500 K)', 'value': 'L'},
                {'label': 'T type star (700-1300 K)', 'value': 'T'},
                {'label': 'Y type star (<700 K)', 'value': 'Y'},
                {'label': 'Wolf-Rayet star (>30000 K)', 'value': 'W'},
                {'label': 'White Dwarf (~5000-100000+ K)', 'value': 'D'},
                {'label': 'Unknown', 'value': 'null'}
            ],
            value=['O', 'B', 'A', 'F', 'G', 'K', 'M', 'null'],  # Default values to filter by
            className="checkbox-container"
        ),
    ]),

    html.Details([
        html.Summary("Yerkes spectral classes"),
        dcc.Checklist(
            id='lum-checklist',
            options=[
                # {'label': 'Hypergiant (0)', 'value': '0'},
                # {'label': 'Supergiant (I)', 'value': 'I'},
                {'label': 'Bright Giant (II)', 'value': 'II'},
                {'label': 'Giant (III)', 'value': 'III'},
                {'label': 'Subgiant (IV)', 'value': 'IV'},
                {'label': 'Main-sequence/Dwarf (V, recommended)', 'value': 'V'},
                {'label': 'Subdwarf (VI)', 'value': 'VI'},
                # {'label': 'White Dwarf (VII)', 'value': 'VII'},
                {'label': 'Unknown', 'value': 'null'}
            ],
            value=['0', 'I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'null'],  # Default values to filter by
            className="checkbox-container"
        ),
    ]),
    
    html.Details([
        html.Summary("Stellar metallicity ratio"),
        dcc.Checklist(
            id='met-checklist',
            options=[
                {'label': 'Iron abundance (recommended)', 'value': '[Fe/H]'},
                {'label': 'General metal content', 'value': '[M/H]'},
                {'label': 'Unknown', 'value': 'null'},
            ],
            value=['[Fe/H]', '[M/H]', 'null'],  # Default values to filter by
            className="checkbox-container"
        ),
    ]), 

    html.H2("Group by:"),
    dcc.RadioItems(
        id="colorcode-radioitems",
        options=[
            {'label': 'Planet Type (Compare Planets only)', 'value': 'pl_type'},
            {'label': 'Harvard Spec. Class', 'value': 'st_teffclass'},
            {'label': 'Yerkes Spec. Class', 'value': 'st_lumclass'},
            {'label': 'Stellar Metallicity', 'value': 'st_metratio'},
            {'label': 'Discovery Method (Compare Planets only)', 'value': 'discoverymethod'},
            {'label': 'Discovery Locale (Compare Planets only)', 'value': 'disc_locale'},
            # {'label': 'Host Star', 'value': 'hostname'}, # too much computational power.
        ],  
        value="pl_type",
        className="checkbox-container"
    ),

    html.H2("How to handle NaN values:"),
    dcc.RadioItems(
        id="nanhandle-radioitems",
        options=[
            # {'label': 'Set to 0', 'value': 'zero'},
            {'label': 'Set to mean', 'value': 'mean'},
            {'label': 'Set to median', 'value': 'median'},
            {'label': 'Remove rows', 'value': 'remove'},
        ],
        value="median",
        className="checkbox-container"
    ),
    
    html.H2("Extra options:"),
    dcc.Checklist(
        id='extra-checklist',
        options=[
            {'label': html.Span(['Simplify planet types and discovery methods (Compare Planets only)'], style={'fontWeight': 'bold'}), 'value': 'simple'},
            {'label': html.Span(['Estimate missing values'], style={'fontWeight': 'bold'}), 'value': 'estimate'},
            {'label': html.Span(['Compare stars'], style={'fontWeight': 'bold'}), 'value': 'suns'},
        ],
        value=[],  # Start unchecked
        className="checkbox-container"
    ),

    html.H2("Min Radius:"),
    dcc.Slider(
        id="min-radius-slider",
        min=0,
        max=10000,
        step=100,
        value=1000,
        marks={i: str(i) for i in range(0, 10001, 1000)},
        tooltip={"placement": "bottom", "always_visible": True}
    ),

    html.Button('Update Graph', id='update-button', n_clicks=1),

    dcc.Tabs(id="tabs", value="tab-1", children=[
        dcc.Tab(label="3D Starmap", value="tab-1", className="tab"),
        dcc.Tab(label="2D Starmap", value="tab-2", className="tab"),
    ]),
    html.Div(id="tabs-content", children=[
        dcc.Graph(id="3d-starmap", figure={}, style={"display": "block"}),
        dcc.Graph(id="2d-starmap", figure={}, style={"display": "none"}),
    ]),  # This will hold the selected graph
    
    html.P('NaN info:', id='naninfo-box'),

    html.P("Data sourced from:"),
    html.Div([
        html.A("https://www.doi.org/10.26133/NEA12", href="https://www.doi.org/10.26133/NEA12", target="_blank"),
        html.Br(),
        html.A("https://www.doi.org/10.26133/NEA36", href="https://www.doi.org/10.26133/NEA36", target="_blank"),
        html.Br(),
        html.A("https://science.nasa.gov/exoplanets/exoplanet-catalog/", href="https://science.nasa.gov/exoplanets/exoplanet-catalog/", target="_blank"),
        html.Br(),
        html.A("https://science.nasa.gov/exoplanets/target-star-catalog/", href="https://science.nasa.gov/exoplanets/target-star-catalog/", target="_blank"),
    ]),
    
    html.Button("Reset Settings", id="reset-button", n_clicks=0),

    dcc.Store(id='settings-store', storage_type='local'),
    dcc.Store(id='rehydration-complete', data=False),
    dcc.Store(id='init-flag', data=False)
], style={'color': 'black', 'font-family': 'Arial', 'backgroundColor': 'white', 'padding': '20px', 'text-align': 'center'})

def update_graph(update, filters, extra, minRad, nanhandle, startoggle, starinput, colorcode, pltype, discmethod, disclocale, teff, lum, met):
    # Load dataset
    if 'estimate' in extra:
        df = pl_es.copy()
    else:
        df = pl_s.copy()

    # Apply filters
    if not 'suns' in extra:
        if 'atmoData' in filters:
            df = df[df['pl_name'].isin(at_s['pl_name']) | df["is_solar"] == True]
        if 'densRange' in filters:
            df=df[(df['pl_dens'] > 1) & (df['pl_dens'] < 5.6)] # Only gets planets with a density between water and metallic iron
        df = df[df['pl_type'].isin(pltype)]  
        if 'simple' in extra:
            df["pl_type"] = df["pl_type"].replace({"super_earth": "terrestrial", "neptune_like": "gas_giant"}) 
            df["discoverymethod"] = df["discoverymethod"].replace({"Transit Timing Variations": "Transit", "Eclipse Timing Variations": "Transit", "Orbital Brightness Modulation": "Transit", "Astrometry": "Radial Velocity", "Pulsation Timing Variations": "Pulsar Timing", "Disc Kinematics": "Imaging"}) 
        if 'null' in discmethod:
            df = df[df['discoverymethod'].isin(discmethod) | df['discoverymethod'].isna()]
        else:
            df = df[df['discoverymethod'].isin(discmethod)]
        if 'null' in disclocale:
            df = df[df['disc_locale'].isin(disclocale) | df['disc_locale'].isna()]
        else:
            df = df[df['disc_locale'].isin(disclocale)]
    if 'null' in teff:
        df = df[df['st_teffclass'].isin(teff) | df['st_teffclass'].isna()] 
    else:
        df = df[df['st_teffclass'].isin(teff)]
    if 'null' in lum:
        df = df[df['st_lumclass'].isin(lum) | df['st_lumclass'].isna()] 
    else:
        df = df[df['st_lumclass'].isin(lum)]
    if 'default' in filters:
        df = df[df['default_flag'] == True]
    if 'noControv' in filters:
        df = df[df['pl_controv_flag'] == False]
    if 'target' in filters:
        df = df[df['hostname'].isin(target_stars) | df["is_solar"] == True] # type: ignore
    if 'null' in met:
        df = df[df['st_metratio'].isin(met) | df['st_metratio'].isna()]
    else:
        df = df[df['st_metratio'].isin(met)]
    if startoggle and starinput.strip():
        # Normalize both sides for safe matching
        df = df[df['hostname'].str.lower() == starinput.strip().lower()]

    # filters out unnecessary columns
    if 'suns' in extra:
        df = df[["sy_dist"] + ["ra"] + ["dec"] +
            ["hostname"] + ["is_solar"] + ["st_spectype"] + ["st_teffclass"] + ["st_lumclass"] + ["st_metratio"]
        ]
        # If there are multiple values for a column, takes the mode (most common value)
        def get_mode(series):
            mode_vals = series.mode()
            if not mode_vals.empty:
                return mode_vals.iloc[0]  # Return the first mode (if multiple)
            return np.nan  # Or return None, depending on preference
        df = df.groupby("hostname", as_index=False).agg(get_mode)
    else:
        df = df[["pl_name"] + ["pl_type"] +
            ["discoverymethod"] + ["disc_refname"] + ["disc_locale"] + ["disc_facility"] + ["disc_telescope"] + ["disc_instrument"] +
            ["hostname"] + ["is_solar"] + ["st_spectype"] + ["st_teffclass"] + ["st_lumclass"] + ["st_metratio"] +
            ["sy_dist"] + ["ra"] + ["dec"]
        ]

    # Returns empty graphs if filtered dataset is empty
    if df.empty:
        return go.Figure(), go.Figure(), "ERROR: filtered dataset is empty"
    if "suns" in extra and colorcode == "pl_type":
        return go.Figure(), go.Figure(), "ERROR: Cannot filter by planet type when using star data."
    
    # Handle NaNs in colorcode by assigning 'unknown'
    df[colorcode] = df[colorcode].fillna("unknown")
    
    # Replace NaNs with the value given by nanhandle
    if nanhandle in ['mean', 'median']:
        cleaned_groups = []
        if colorcode == "PC":
            df_sub = df.copy()
            fill_vals = (
                df_sub['sy_dist'].mean() if nanhandle == 'mean'
                else df_sub['sy_dist'].median()
            )
            if pd.isna(fill_vals):
                fill_vals = 0
            df_sub['sy_dist'] = df_sub['sy_dist'].fillna(fill_vals)
            cleaned_groups.append(df_sub)

        else:
            for group_val, df_group in df.groupby(colorcode):
                df_sub = df_group.copy()
                fill_vals = (
                    df_sub['sy_dist'].mean() if nanhandle == 'mean'
                    else df_sub['sy_dist'].median()
                )
                if pd.isna(fill_vals):
                    fill_vals = 0
                df_sub['sy_dist'] = df_sub['sy_dist'].fillna(fill_vals)
                cleaned_groups.append(df_sub)

        df = pd.concat(cleaned_groups, ignore_index=True)
    else:
        df = df.dropna(subset='sy_dist')
    naninfo = f"Total entries: {len(df)}"

    # If distance is below minRad, sets distance to minRad on the map (doesn't change the data)
    dist = df['sy_dist'] if minRad == 0 else np.where(df['sy_dist'] <= minRad, 1, df['sy_dist'] / minRad)
    # Convert spherical coordinates (RA, Dec, Distance) to Cartesian (X, Y, Z)
    df['x'] = dist * np.cos(np.radians(df['dec'])) * np.cos(np.radians(df['ra']))
    df['y'] = dist * np.cos(np.radians(df['dec'])) * np.sin(np.radians(df['ra']))
    df['z'] = dist * np.sin(np.radians(df['dec']))
    
    fig2d = go.Figure()
    fig3d = go.Figure()

    # Color legend    
    color_map = {
        "terrestrial": "blue", "super_earth": "red", "neptune_like": "green", "gas_giant": "purple", "tba": "black",
        "II": "yellow", "III": "orange", "IV": "green", "V": "blue", "VI": "black",
        "B": "darkblue", "A": "royalblue", "F": "seagreen", "G": "yellow", "K": "orange", "M": "red",
        "Transit": "royalblue", "Transit Timing Variations": "slateblue", "Eclipse Timing Variations": "mediumslateblue", "Orbital Brightness Modulation": "cornflowerblue", "Radial Velocity": "crimson", "Astrometry": "firebrick", "Imaging": "seagreen", "Disc Kinematics": "mediumseagreen", "Microlensing": "darkgoldenrod", "Pulsar Timing": "mediumvioletred", "Pulsation Timing Variations": "orchid", "Known Since Antiquity": "black",
        "Space": "deepskyblue", "Ground": "sienna", "Multiple Locales": "mediumorchid",
        "[Fe/H]": "steelblue", "[M/H]": "darkcyan",
        "unknown": "gray",
    }
    if colorcode in df.columns and df[colorcode].dtype == "object":
        # Count values
        label_counts = df[colorcode].value_counts().to_dict()

        # Create a mapping: e.g., "terrestrial" → "terrestrial (42)"
        labeled_with_counts = {
            key: f"{key} ({label_counts.get(key, 0)})" for key in df[colorcode].unique()
        }

        # Apply the relabeling
        df["colorcode_labeled"] = df[colorcode].map(labeled_with_counts)
        colorcode_use = "colorcode_labeled"

        # Generate fallback color iterator from Plotly palette
        default_colors = px.colors.qualitative.Plotly
        default_color_cycle = iter(default_colors)

        color_map_labeled = {}
        for key in df[colorcode].unique():
            label = labeled_with_counts[key]
            if key in color_map:
                color_map_labeled[label] = color_map[key]
            else:
                color_map_labeled[label] = next(default_color_cycle)
    else:
        # For continuous or numeric colorcodes, no relabeling
        colorcode_use = colorcode
        color_map_labeled = color_map

    # marker size logic
    dmin = df['sy_dist'].min()
    dmax = df['sy_dist'].max()
    minSize = 2
    maxSize = 5
    if dmax != dmin:
        df['marker_size'] = minSize + (df['sy_dist'] - dmin) / (dmax - dmin) * (maxSize - minSize)
    else:
        df['marker_size'] = (minSize+maxSize)/2  # fallback size
    sizeref_val = 2. * maxSize / (maxSize**2)

    # Create hover text            
    df['hover_text'] = (
        "star Name: " + df['hostname'] +
        "<br>RA: " + df['ra'].round(2).astype(str) + "°, Dec: " + df['dec'].round(2).astype(str) + "°" +
        "<br>Dist. from Earth: " + df['sy_dist'].round(2).astype(str) + " pc"
    ) if "suns" in extra else (
        "Planet Name: " + df['pl_name'] +
        "<br>Planet Type: " + df['pl_type'] +
        "<br>Host Star: " + df['hostname'] +
        "<br>RA: " + df['ra'].round(2).astype(str) + "°, Dec: " + df['dec'].round(2).astype(str) + "°" +
        "<br>Dist. from Earth: " + df['sy_dist'].round(2).astype(str) + " pc" +
        "<br>Disc. Method: " + df['discoverymethod'] +
        "<br>Disc. Reference: " + df['disc_refname'] +
        "<br>Disc. Locale: " + df['disc_locale'] +
        "<br>Disc. Facility: " + df['disc_facility'] +
        "<br>Disc. Telescope: " + df['disc_telescope'] +
        "<br>Disc. Instrument: " + df['disc_instrument']
    )

    # Plotting Earth
    fig3d.add_trace(go.Scatter3d(
        x=[0],
        y=[0],
        z=[0],
        mode="markers",
        marker=dict(
            size=10,  # Make Earth larger
            color="white",
            line=dict(color="royalblue", width=4)
        ),
        text=["Planet Name: Earth<br>Planet Type: terrestrial"],  # ← put your custom HTML hover text here
        hoverinfo='text',  # ← tell Plotly to use 'text' for hover display
        name="Earth"
    ))

    # plotting everything else
    for category, color in color_map_labeled.items():
        filtered_df = df[(df[colorcode_use] == category)]
        if filtered_df.empty:
            continue
        sizes = filtered_df['marker_size']
        hover = filtered_df['hover_text']
        fig3d.add_trace(go.Scatter3d(
            x=filtered_df["x"],
            y=filtered_df["y"],
            z=filtered_df["z"],
            mode="markers",
            marker=dict(size=2, color=color),
            name=category,
            text=hover,
            hoverinfo='text'
        ))
        fig2d.add_trace(go.Scatter(
            x=filtered_df["ra"],
            y=filtered_df["dec"],
            mode='markers',
            name=category,
            marker=dict(
                size=sizes,
                color=color,
                sizemode='diameter',
                sizeref=sizeref_val,
                sizemin=minSize,
                line=dict(width=0)
            ),
            text=hover,
            hoverinfo='text'
        ))
    
    fig2d.update_layout(
        legend=dict(
            font=dict(size=20),  # Adjust size here
            itemsizing='constant'  # Keeps marker size consistent
        )
    )
    fig2d.update_xaxes(title_text="RA (degrees)")
    fig2d.update_yaxes(title_text="Dec (degrees)")
    # Improve legend visibility and title
    fig3d.update_layout(
        legend=dict(
            x=0, y=1,  # Positioning
            title="Planet Type",
            bgcolor="rgba(255, 255, 255, 0.5)",  # Semi-transparent background
            borderwidth=1,
            font=dict(size=20),  # Adjust size here
            itemsizing='constant'  # Keeps marker size consistent
        ),
        scene=dict(
            xaxis=dict(showgrid=False, showticklabels=False, visible=False),
            yaxis=dict(showgrid=False, showticklabels=False, visible=False),
            zaxis=dict(showgrid=False, showticklabels=False, visible=False),
            annotations=[],
            aspectmode="data"
        ),
        scene_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        margin=dict(l=0, r=0, b=0, t=0)
    )
    fig3d.update_traces(
        hoverlabel=dict(
            font=dict(color="black"),
            bgcolor="white",
        ),
    )

    return fig3d, fig2d, html.Pre(naninfo)

# changes displayed graph based on selected tab
@app.callback(
    [Output('3d-starmap', 'style'),
     Output('2d-starmap', 'style'),],
    [Input("tabs", "value")]
)
def update_tabs(tab):
    styles = {
        "tab-1": [{"display": "block"}, {"display": "none"}],
        "tab-2": [{"display": "none"}, {"display": "block"}],
    }
    return styles.get(tab, [{"display": "block"}, {"display": "none"}])

# updates graphs based on user input or stored settings
@app.callback(
    [Output('3d-starmap', 'figure'),
     Output('2d-starmap', 'figure'),
     Output('naninfo-box', 'children'),
     Output('settings-store', 'data')],
    [Input('update-button', 'n_clicks'),
     Input('settings-store', 'data')],
    [State('filter-checklist', 'value'),
     State('extra-checklist', 'value'),
     State('min-radius-slider', 'value'),
     State('nanhandle-radioitems', 'value'),
     State('host-star-toggle', 'value'),
     State('host-star-input', 'value'),
     State('colorcode-radioitems', 'value'),
     State('pltype-checklist', 'value'),
     State('discmethod-checklist', 'value'),
     State('disclocale-checklist', 'value'),
     State('teff-checklist', 'value'),
     State('lum-checklist', 'value'),
     State('met-checklist', 'value'),]
)
def update_figures(n_clicks, settings, *states):
    # Check if the callback was triggered by a button click or settings change
    ctx = dash.callback_context
    if not ctx.triggered:
        raise dash.exceptions.PreventUpdate

    trigger = ctx.triggered[0]['prop_id']

    if trigger == 'update-button.n_clicks':
        # Use current inputs
        fig3d, fig2d, naninfo = update_graph(n_clicks, *states)
        settings_dict = dict(zip([
            'filter', 'extra', 'minRad', 'nanhandle', 'hosttoggle', 'hostinput', 
            'colorcode', 'pltype', 'discmethod', 'disclocale', 'teff', 'lum', 'met'
        ], states))
        return fig3d, fig2d, naninfo, settings_dict

    elif trigger == 'settings-store.data':
        # Use stored settings, not states
        if not settings:
            raise dash.exceptions.PreventUpdate
        fig3d, fig2d, naninfo = update_graph(0, *unpack_settings(settings))
        return fig3d, fig2d, naninfo, dash.no_update

# Restore settings from local storage when the app starts
@app.callback(
    [Output('filter-checklist', 'value'),
     Output('extra-checklist', 'value'),
     Output('min-radius-slider', 'value'),
     Output('nanhandle-radioitems', 'value'),
     Output('host-star-toggle', 'value'),
     Output('host-star-input', 'value'),
     Output('colorcode-radioitems', 'value'),
     Output('pltype-checklist', 'value'),
     Output('discmethod-checklist', 'value'),
     Output('disclocale-checklist', 'value'),
     Output('teff-checklist', 'value'),
     Output('lum-checklist', 'value'),
     Output('met-checklist', 'value'),
     Output('init-flag', 'data')],
    Input('settings-store', 'data'),
    State('init-flag', 'data'),
    prevent_initial_call="initial_duplicate"
)
def restore_settings(settings, already_initialized):
    if not settings or already_initialized:
        raise dash.exceptions.PreventUpdate
    return (
        settings['filter'], settings['extra'], settings['minRad'], settings['nanhandle'],
        settings['hosttoggle'], settings['hostinput'], settings['colorcode'],
        settings['pltype'], settings['discmethod'], settings['disclocale'],
        settings['teff'], settings['lum'], settings['met'], True
    )

# Unpack settings from the stored dictionary
def unpack_settings(settings):
    return (
        settings['filter'], settings['extra'], settings['minRad'], settings['nanhandle'],
        settings['hosttoggle'], settings['hostinput'], settings['colorcode'],
        settings['pltype'], settings['discmethod'], settings['disclocale'],
        settings['teff'], settings['lum'], settings['met'],
    )

# Clear the store when the reset button is clicked
@app.callback(
    Output('settings-store', 'data', allow_duplicate=True),
    Input('reset-button', 'n_clicks'),
    prevent_initial_call=True
)
def clear_store(n):
    return None  # or {}

if __name__ == '__main__':
    app.run(debug=True, port=8055)
    print("running on localhost:8055")

    # get bargraphs with distribution of both terrestrials only and all planets